<a href="https://colab.research.google.com/github/UMB200/pytorch_projects/blob/main/05_pytorch_going_modular_cell_mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.0 Get data

### 1.1 Import necessary libariaries

In [ ]:
import os
import requests
import zipfile
from pathlib import Path

### 1.2 Get data

In [ ]:
# Setup path to data folder
data_path = Path("data_path/")
img_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exists, download it and prepare it
if img_path.is_dir():
  print(f"{img_path} directory already exists")
else:
  print(f"Creating directory {img_path}")
  img_path.mkdir(parents=True, exist_ok=True)
# Download data
url_to_download = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
file_path = data_path / "pizza_steak_sushi.zip"
with open(file_path, "wb") as f:
  print(f"Downloading data")
  f.write(requests.get(url_to_download).content)
#Unzip data
with zipfile.ZipFile(file_path, "r") as zip_ref:
  print(f"Unzipping: {file_path}")
  zip_ref.extractall(img_path)

# Remove zip file
os.remove(file_path)

Creating directory data_path/pizza_steak_sushi
Unzipping: data_path/pizza_steak_sushi.zip


In [ ]:
# Setup training and testing paths
train_dir = img_path / "train"
test_dir = img_path / "test"

train_dir, test_dir

(PosixPath('data_path/pizza_steak_sushi/train'),
 PosixPath('data_path/pizza_steak_sushi/test'))

## 2.0 Create Datasets & DataLoaders

Convert code to script: `%%writefile going_modular/data_setup.py`

In [ ]:
# Transform images to tensors
from torchvision import transforms
img_to_tensor_transform = transforms.Compose([transforms.Resize(size=(64, 64)),
                                     transforms.ToTensor()])

In [ ]:
import os
os.makedirs("going_modular", exist_ok=True)

In [ ]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """

  # Use ImageFolder to create dataset(s)
  train_dataset = datasets.ImageFolder(train_dir, transform=transform)
  test_dataset = datasets.ImageFolder(test_dir,transform=transform)

  # Get class names
  class_names = train_dataset.classes

  # Turn datasets to DataLoaders
  train_dataloader = DataLoader(train_dataset,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=True,
                                pin_memory=True)
  test_dataloader = DataLoader(dataset=test_dataset,
                               batch_size=batch_size,
                               num_workers=num_workers,
                               shuffle=False,
                               pin_memory=True)

  return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [ ]:
from going_modular import data_setup
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir,
    test_dir,
    img_to_tensor_transform,
    batch_size = 32)

## 3.0 Making a model

In [ ]:
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x77fc85d07610>,
 ['pizza', 'steak', 'sushi'])

In [ ]:
%%writefile going_modular/model_builder.py
"""
    Model architecture copying TinyVGG from:
    https://poloclub.github.io/cnn-explainer/
    Contains PyTorch model code to instantiate a TinyVGG model.
"""

import torch
from torch import nn

class Model_Builder_TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
      input_shape: An integer indicating number of input channels.
      hidden_units: An integer indicating number of hidden units between layers.
      output_shape: An integer indicating number of output units.
      num_of_units: An integer indicating image size downsamples
  """


  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int,
               num_of_units: int) -> None:
               super().__init__()
               self.cnn_block_1 = nn.Sequential(
                   nn.Conv2d(in_channels=input_shape,
                             out_channels=hidden_units,
                             kernel_size=3,
                             stride=1,
                             padding=1),
                   nn.ReLU(),
                   nn.Conv2d(in_channels=hidden_units,
                             out_channels=hidden_units,
                             kernel_size=3,
                             stride=1,
                             padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=2,
                                stride=2)
               )
               self.cnn_block_2 = nn.Sequential(
                   nn.Conv2d(hidden_units, hidden_units, 3, 1, 1),
                   nn.ReLU(),
                   nn.Conv2d(hidden_units, hidden_units, 3, 1, 1),
                   nn.ReLU(),
                   nn.MaxPool2d(2)
               )
               self.classifier = nn.Sequential(
                   nn.Flatten(),
                   nn.Linear(in_features=hidden_units*num_of_units*num_of_units,
                             out_features=output_shape)
               )
  def forward(self, x: torch.Tensor):
    return self.classifier(self.cnn_block_2(self.cnn_block_1(x)))

Writing going_modular/model_builder.py


In [ ]:
from going_modular import model_builder
import torch
device  = "cuda" if torch.cuda.is_available() else "cpu"

model_1 = model_builder.Model_Builder_TinyVGG(input_shape=3,
                                             hidden_units=10,
                                             output_shape=len(class_names),
                                             num_of_units=13).to(device)
model_1

Model_Builder_TinyVGG(
  (cnn_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cnn_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

## 4.0 Create `train_step()` and `test_step()` and `train()` to combine them

### 4.1 Create `engine()`

In [ ]:
%%writefile going_modular/engine.py

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from torch import nn

def train_step_loop(model: torch.nn.Module,
                    data_loader: torch.utils.data.DataLoader,
                    loss_fn: torch.nn.Module,
                    optimizer: torch.optim.Optimizer,
                    device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  model.train()
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(data_loader):
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  return train_loss, train_acc

def test_step_loop(model: torch.nn.Module,
                    data_loader: torch.utils.data.DataLoader,
                   loss_fn: torch.nn.Module,
                   device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  model.eval()
  test_loss, test_acc = 0, 0
  with torch.inference_mode():
    for batch, (X, y) in enumerate(data_loader):
      X, y = X.to(device), y.to(device)
      test_pred_logits = model(X)

      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += (test_pred_labels == y).sum().item()/len(test_pred_labels)
  test_loss /= len(data_loader)
  test_acc /= len(data_loader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
               train_dataloader: torch.utils.data.DataLoader,
               test_dataloader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               loss_fn: torch.nn.Module,
               epochs: int,
               device: torch.device)-> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step_loop(model=model,
                                            data_loader=train_dataloader,
                                            loss_fn=loss_fn,
                                            optimizer=optimizer,
                                            device=device)
    test_loss, test_acc = test_step_loop(model=model,
                                        data_loader=test_dataloader,
                                        loss_fn=loss_fn,
                                         device=device)
    print(f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}")
    get_tensor_val = lambda x: x.item() if isinstance(x, torch.Tensor) else x
    results["train_loss"].append(get_tensor_val(train_loss))
    results["train_acc"].append(get_tensor_val(train_acc))
    results["test_loss"].append(get_tensor_val(test_loss))
    results["test_acc"].append(get_tensor_val(test_acc))
  return results

Overwriting going_modular/engine.py


In [ ]:
from going_modular import engine
engine

<module 'going_modular.engine' from '/content/going_modular/engine.py'>

## 5.0 Create a function to save the model `utils.py`

In [ ]:
%%writefile going_modular/utils.py

import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


## 6.0 Train, evaluate and save the model `train.py`

In [ ]:
%%writefile going_modular/train.py
"""
Trains Python model for image classification using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils
from torchvision import transforms
from timeit import default_timer as timer
import argparse


# SETUP HYPERPARAMETERS
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001
NUM_UNITS = 16

# SETUP DIRECTORIES
train_dir_scripted = "data_path/pizza_steak_sushi/train"
test_dir_scripted =  "data_path/pizza_steak_sushi/test"

# SETUP TARGET DEVICE
device = "cuda" if torch.cuda.is_available() else "cpu"

# CREATE TRANSFORMS
data_transform = transforms.Compose([transforms.Resize(size=(64, 64)),
                                     transforms.ToTensor()])

# CREATE DATALOADERS USING data_setup.py
train_dataloader_scripted, test_dataloader_scripted, class_names_scripted = data_setup.create_dataloaders(
    train_dir = train_dir_scripted,
    test_dir = test_dir_scripted,
    transform = data_transform,
    batch_size = BATCH_SIZE)

# CREATE MODEL USING model_builder.py
model_saved = model_builder.Model_Builder_TinyVGG(input_shape=3,
                                             hidden_units=HIDDEN_UNITS,
                                             output_shape=len(class_names_scripted),
                                             num_of_units=NUM_UNITS).to(device)

# SETUP LOSS_FN AND OPTIMIZER
loss_fn_cross_entropy = torch.nn.CrossEntropyLoss()
optimizer_adam = torch.optim.Adam(model_saved.parameters(), lr=LEARNING_RATE)

# START TRAINING USING engine.py
star_time = timer()
engine.train(model=model_saved,
             train_dataloader = train_dataloader_scripted,
             test_dataloader = test_dataloader_scripted,
             loss_fn = loss_fn_cross_entropy,
             optimizer = optimizer_adam,
             epochs = NUM_EPOCHS,
             device = device)
end_time = timer()
print(f"[INFO] Total training time: {end_time-star_time:.3f} seconds")

# SAVE MODEL USING utils.py
utils.save_model(model = model_saved,
                 target_dir = "models",
                 model_name = "05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting going_modular/train.py


In [ ]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1042 | train_acc: 0.3984 | test_loss: 1.1132 | test_acc: 0.2604
 20% 1/5 [00:02<00:08,  2.23s/it]Epoch: 2 | train_loss: 1.1127 | train_acc: 0.3047 | test_loss: 1.1189 | test_acc: 0.2604
 40% 2/5 [00:05<00:07,  2.58s/it]Epoch: 3 | train_loss: 1.0854 | train_acc: 0.4375 | test_loss: 1.0639 | test_acc: 0.5417
 60% 3/5 [00:07<00:05,  2.71s/it]Epoch: 4 | train_loss: 1.0953 | train_acc: 0.2812 | test_loss: 1.0377 | test_acc: 0.5521
 80% 4/5 [00:09<00:02,  2.44s/it]Epoch: 5 | train_loss: 1.0860 | train_acc: 0.3867 | test_loss: 1.0550 | test_acc: 0.5322
100% 5/5 [00:12<00:00,  2.43s/it]
[INFO] Total training time: 12.154 seconds
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth
